# Canonical ensemble Monte Carlo


In this example, a low density simulation is conducted with a constant number of particles.
The average energy is compared to published results from the NIST SRSW.
https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website

The log file is designed to monitor simulations but may not be the best route to computing properties.
For example, the energy is output in the log file, but this tutorial shows two alternate ways to compute average energy.
First, an [Analyze](../../doc/Analyze.rst), [Energy](../../../steppers/doc/Energy.rst), to compute average energies.
Second, the Monte Carlo simulation attempts were performed one step at a time to allow for [accumulation](../../../math/doc/Accumulator.rst) of the ensemble average directly in the Python script.

In [1]:
import sys
import subprocess
import unittest
import feasst as fst

def read_checkpoint(filename):                                         
    """Return contents of checkpoint file as a string"""               
    with open (filename, "r") as myfile:                               
        checkpoint=myfile.readlines()                                  
    assert(len(checkpoint) == 1)  # checkpoint files should have only one line  
    return checkpoint[0]

class TestMonteCarlo1LJNVT(unittest.TestCase):
    """Test a canonical ensemble Lennard Jones Monte Carlo simulation"""
    def test_srsw(self, num_particles=500, density=0.001, trials_per=1e5):
        """Compare with the reported average energy from the NIST SRSW.
        https://mmlapps.nist.gov/srs/LJ_PURE/mc.htm
        https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website

        num_particles -- number of LJ particles
        density -- number density
        trials_per -- steps between each Anaylze/Modify
        """
            
        monte_carlo = fst.MonteCarlo()
        # monte_carlo.set(fst.MakeRandomMT19937(fst.args({"seed": "1234"})))
        monte_carlo.add(fst.MakeConfiguration(fst.args({
            "cubic_side_length": str((num_particles/density)**(1./3.)),
            "particle_type0": fst.install_dir() + "/forcefield/lj.fstprt"})))
        monte_carlo.add(fst.MakePotential(fst.MakeLennardJones()))
        monte_carlo.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
        monte_carlo.set(fst.MakeThermoParams(fst.args({"beta": str(1./0.9), "chemical_potential": "1."})))
        monte_carlo.set(fst.MakeMetropolis())
        monte_carlo.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "2."})))
        monte_carlo.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
        monte_carlo.run(fst.MakeRun(fst.args({"until_num_particles": str(num_particles)})))
        monte_carlo.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
        monte_carlo.add(fst.MakeLogAndMovie(fst.args({
            "trials_per" : str(trials_per),
            "file_name": "movie",
            "clear_file": "true"})))
        monte_carlo.add(fst.MakeCheckEnergy(fst.args({"trials_per" : str(trials_per)})))
        monte_carlo.add(fst.MakeTune())

        # equilibrate
        monte_carlo.attempt(int(1e7))

        # compute average energy using a stepper/analysis and output into file
        energy = fst.MakeEnergy(fst.args({
            "trials_per_update": "1",
            "trials_per_write": str(trials_per),
            "file_name": "lj_nvt_srsw_energy.txt",
        }))
        monte_carlo.add(energy)

        # compute average using this script
        energy_alt = fst.Accumulator()

        # production
        for _ in range(int(1e7)):
            monte_carlo.attempt()
            energy_alt.accumulate(monte_carlo.criteria().current_energy())

        # test that the two methods to compute average energy are the same
        self.assertAlmostEqual(energy.energy().average(), energy_alt.average(), delta=1e-6)

        # test the average against the NIST SRSW
        stdev = (energy.energy().block_stdev()**2 + (1.89E-05)**2)**(1./2.)
        # print(energy.energy().average(), energy_alt.average())
        self.assertAlmostEqual(-9.9165E-03*num_particles, energy.energy().average(),
                              delta=2.576*stdev)

    def test_srsw_alt(self, num_particles=500, density=0.001, trials_per=1e5, beta=1./0.9,
                      fstprt=fst.install_dir() + "/forcefield/lj.fstprt"):
        "Same as above, but using an input text file and testing result from checkpoint"
        params = {"box_length": (num_particles/density)**(1./3.)}
        params = dict(locals(), **params)
        fst_input = """
Checkpoint file_name checkpoint.fst
RandomMT19937 seed time
Configuration cubic_side_length {box_length} particle_type {fstprt}
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
ThermoParams beta 0.1 chemical_potential 10
Metropolis
TrialTranslate tunable_param 2. tunable_target_acceptance 0.2
TrialAdd particle_type 0
Run until_num_particles {num_particles}
RemoveTrial name TrialAdd
ThermoParams beta {beta}
Tune trials_per {trials_per}
CheckEnergy trials_per {trials_per} tolerance 1e-8
Run num_attempts 1e6
RemoveModify name Tune
Log trials_per {trials_per} file_name lj.txt
Energy trials_per_write {trials_per} file_name en.txt
Run num_attempts 1e6
WriteCheckpoint
""".format(**params)
        with open('tutorial_1_lj_nvt.txt', 'w') as fsttxt:
            fsttxt.write(fst_input)
        subprocess.call(fst.install_dir() + "/build/bin/fst < tutorial_1_lj_nvt.txt")
        
        # test the average against the NIST SRSW by deserializing the checkpoint
        mc = fst.MonteCarlo().deserialize(read_checkpoint('checkpoint.fst'))
        energy = fst.SeekAnalyze().reference("Energy", mc).accumulator()
        stdev = (energy.block_stdev()**2 + (1.89E-05)**2)**(1./2.)
        self.assertAlmostEqual(-9.9165E-03*num_particles, energy.average(),
                               delta=2.576*stdev)

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [2]:
%time  # Note: any line starting with % is only to be used with ipynb
unittest.main(argv=[''], verbosity=2, exit=False)

test_srsw (__main__.TestMonteCarlo1LJNVT)
Compare with the reported average energy from the NIST SRSW. ... 

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs


ok
test_srsw_alt (__main__.TestMonteCarlo1LJNVT)
Same as above, but using an input text file and testing result from checkpoint ... ok

----------------------------------------------------------------------
Ran 2 tests in 1258.481s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../../CONTACT.rst) us. Any feedback is appreciated!